In [ ]:
import os

In [ ]:
username = os.environ.get("USER") or os.environ.get("USERNAME")
if username:
    print(f"Username: {username}")
else:
    print("Username not found in environment variables.")
    sys.exit()

In [ ]:
from pathlib import Path
cwd = str(Path.cwd())
print(f"PWD: {cwd}")
os.environ["PYTHONPATH"] += f":{cwd}"
os.environ["X509_USER_PROXY"] = f"{cwd}/voms_proxy.txt"
os.environ['XRD_REQUESTTIMEOUT'] = "2400"
!echo $PYTHONPATH

In [ ]:
from dask_gateway import Gateway
# gateway = Gateway()
gateway = Gateway(
    "http://dask-gateway-k8s.geddes.rcac.purdue.edu/",
    proxy_address="traefik-dask-gateway-k8s.cms.geddes.rcac.purdue.edu:8786",
)

In [ ]:
for cluster_info in gateway.list_clusters():
    print(cluster_info)

In [ ]:
cluster = gateway.new_cluster(
    # conda_env = "/depot/cms/kernels/coffea2024",
    # conda_env = "/depot/cms/kernels/root632",
    # conda_env = "/depot/cms/kernels/coffea_latest",
    conda_env = "/depot/cms/users/yun79/conda_envs/yun_coffea_latest",
    worker_cores = 2, #2
    worker_memory = 10, #6
    env = dict(os.environ),
)

cluster.adapt(minimum=3, maximum=499) # 399 for low resources, otherwise 199
cluster
# cluster.adapt(100) 

In [ ]:
for cluster_info in gateway.list_clusters():
    print(cluster_info)

In [ ]:
for cluster_info in gateway.list_clusters():
    # print(cluster_info.name)
    # if "aa072b58804d41119254e8a33652223c" not in cluster_info.name:
    if "dac3e4199e64482ab41b6e9bb42c1d4a" in cluster_info.name:
        print(cluster_info.name)
        gateway.connect(cluster_info.name).shutdown()


In [ ]:
"""
To shutdown the cluster, run this cell
Equivalently, you can also do "squeue -u <username>" 
if this is the only slurm job you're doing
"""

for cluster_info in gateway.list_clusters():
    gateway.connect(cluster_info.name).shutdown()

In [ ]:
for cluster_info in gateway.list_clusters():
    print(cluster_info)

In [ ]:
options = gateway.cluster_options()  

In [ ]:
# options.worker_cores
print(options)

In [ ]:
! squeue -u shar1172